# Import Libraries

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

In [2]:
torch.manual_seed(24)
np.random.seed(24)

# Import Dataset

In [3]:
train_df = pd.read_csv('HW2_data/Q3_train.csv')
test_df = pd.read_csv('HW2_data/Q3_test.csv')
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [4]:
num_classes = len(np.unique(train_df['label']))
num_classes

24

It doesn't contain one of letters. So we will map them to numbers 0 to 23.

In [5]:
english_letters = [chr(ord('A') + i) for i in range(26)]
unique_labels = np.unique(np.unique(train_df['label']))
idx_to_letter = {i: english_letters[num] for i, num in enumerate(unique_labels)}
label_to_idx = {num:i for i, num in enumerate(unique_labels)}
idx_to_letter

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'K',
 10: 'L',
 11: 'M',
 12: 'N',
 13: 'O',
 14: 'P',
 15: 'Q',
 16: 'R',
 17: 'S',
 18: 'T',
 19: 'U',
 20: 'V',
 21: 'W',
 22: 'X',
 23: 'Y'}

In [ ]:
train_x = train_df.drop(columns=['label']).to_numpy()
test_x = test_df.drop(columns=['label']).to_numpy()
train_y = train_df['label'].map(label_to_idx).to_numpy()
test_y = test_df['label'].map(label_to_idx).to_numpy()

# Visualization

In [ ]:
n_in = train_x.shape[1]
sqrt = np.sqrt(n_in).astype(int)
image_shape = (sqrt, sqrt)
image_shape

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
axes = axes.flat
for ax in axes:
    i = np.random.randint(0, len(train_x))
    x, y = train_x[i], train_y[i]
    img = x.reshape(image_shape)
    ax.imshow(img)
    ax.set_title(f'Letter {idx_to_letter[y]}')
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout(pad=3)
plt.show()

# Dataset and Dataloaders

In [ ]:
class HandLetters(torch.utils.data.Dataset):

    def __init__(self, x, y):
        self.features = torch.tensor(x).type(torch.FloatTensor)
        self.target = torch.tensor(y).type(torch.LongTensor)

    def __len__(self):
        return len(self.target)

    def __getitem__(self, item):
        x = self.features[item]
        y = self.target[item]
        return x, y

In [ ]:
train_set = HandLetters(train_x, train_y)
test_set = HandLetters(test_x, test_y)

Spliting Data:

In [ ]:
from torch.utils.data import random_split
train_len = int(len(train_set) * 0.8)
val_len = len(train_set) - train_len
train_set, val_set = random_split(train_set, [train_len, val_len])

Dataloaders:

In [ ]:
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)

# Defining Model

In [ ]:
from more_itertools import pairwise

class HandLettersDetector(nn.Module):

    def __init__(self, layers_inputs, dropout=False):
        super().__init__()
        layers_shape = list(pairwise(layers_inputs))
        layers = []
        for shape in layers_shape[:-1]:
            layers.append(nn.Linear(*shape))
            layers.append(nn.ReLU())
            if dropout:
                layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(*layers_shape[-1]))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        return self.layers(x)

In [ ]:
layers_inputs = [n_in, 512, 256, 128, 64, num_classes]
model = HandLettersDetector(layers_inputs)
model

# Training Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def accuracy(y_pred: np.ndarray, y_true: np.ndarray):
    return (y_true == y_pred).mean()

In [ ]:
def one_epoch(model, loader, criterion, optimizer=None, epoch=None, train=True, set_name='Train'):
    total_loss = 0
    N = len(loader.dataset)
    Y = []
    Y_pred = []
    if train:
        model.train()
    else:
        model.eval()

    with torch.set_grad_enabled(train), tqdm.tqdm(enumerate(loader)) as pbar:
        for i, (x, y) in pbar:
            if train:
                optimizer.zero_grad()

            x = x.to(device)
            y = y.to(device)
            p = model(x)

            loss = criterion(p, y)

            total_loss += loss.item() * len(x)
            pbar.set_description(f'{epoch}: {set_name} Loss: {total_loss / N:.3e}')
            if train:
                loss.backward()
                optimizer.step()

            y_pred = p.argmax(dim=-1)
            Y.append(y.cpu().numpy())
            Y_pred.append(y_pred.cpu().numpy())

    total_loss /= N

    Y = np.concatenate(Y)
    Y_pred = np.concatenate(Y_pred)
    acc = accuracy(Y_pred, Y)
    print(f'Accuracy of {set_name} set: {acc}')
    return total_loss, acc

In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [ ]:
def train_model(model, dataloaders, num_epochs, criterion, optimizer, model_name='pytroch-model'):
    train_loader, val_loader = dataloaders
    min_val_loss = np.inf
    for epoch in range(num_epochs):
        train_loss, train_acc = one_epoch(model, train_loader, criterion, optimizer, epoch, train=True, set_name='Train')
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_loss, val_acc = one_epoch(model, val_loader, criterion, epoch=epoch, train=False, set_name='Validation')
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        print('\n', '-' * 60)
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            torch.save(model.state_dict(), f'{model_name}.pt')


    plt.plot(train_losses, label='train')
    plt.plot(val_losses, label='val')
    plt.title('loss history of training and val sets')
    plt.legend()
    plt.show()

    plt.plot(train_accuracies, label='train')
    plt.plot(val_accuracies, label='val')
    plt.title('Accuracy history of training and val sets')
    plt.legend()
    plt.show()

    model.load_state_dict(torch.load(f'{model_name}.pt'))
    return model, min_val_loss

Training with SGD:

In [ ]:
train_losses.clear()
val_losses.clear()
train_accuracies.clear()
val_accuracies.clear()

# Training config
lr = 0.001
model = HandLettersDetector(layers_inputs).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)
model, min_val_loss = train_model(model, [train_loader, val_loader], 25, criterion, optimizer)

In [ ]:
min_val_loss

In [ ]:
test_loss, test_acc = one_epoch(model, test_loader, criterion, train=False, set_name='Test')

Training with Adam:

In [ ]:
train_losses.clear()
val_losses.clear()
train_accuracies.clear()
val_accuracies.clear()

# Training config
lr = 0.0001
model = HandLettersDetector(layers_inputs).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
model, min_val_loss = train_model(model, [train_loader, val_loader], 25, criterion, optimizer)

In [ ]:
min_val_loss

In [ ]:
test_loss, test_acc = one_epoch(model, test_loader, criterion, train=False, set_name='Test')

Training with Dropout:

In [ ]:
train_losses.clear()
val_losses.clear()
train_accuracies.clear()
val_accuracies.clear()

# Training config
lr = 8e-5
model = HandLettersDetector(layers_inputs, dropout=0.25).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
model, min_val_loss = train_model(model, [train_loader, val_loader], 25, criterion, optimizer)

In [ ]:
min_val_loss

In [ ]:
test_loss, test_acc = one_epoch(model, test_loader, criterion, train=False, set_name='Test')